In [ ]:
!pip install tqdm --quiet

In [ ]:
import requests
from tqdm.notebook import tqdm
import os.path
from calendar import monthrange
import time

API_KEY = "FpIG19BfekWUFWGPhb5Nl_zIMPA7Z7gI7ZQ1DESEGkk"
# API_KEY = "zw_mUBWe3F72OcpgY1j2FjnVQMULgW52uOJzvvdxT6A"

# Note the months to fetch and generate dates
months = [3, 4]
dates = [f"""2022-{month:02}-{(day+1):02}""" for month in months for day in range(monthrange(2022, month)[1])]

# Show last 10 days to fetch
print(len(dates))
dates[-10:]

In [ ]:
# Downloading real-time historical data.
!mkdir -p data/0-koda-gtfs-rt/vp/
!mkdir -p data/0-koda-gtfs-rt/tu/

headers = {'accept': 'application/octet-stream'}
for date in tqdm(dates):

    # Create parameter dictionary including key for this iteration
    params = {'date': date, 'key': API_KEY}

    # Construct filename and output path.
    # If files are already downloaded, we skip this iteration.
    file_path_vp = f"""./data/0-koda-gtfs-rt/vp/{params['date']}.7z"""
    file_path_tu = f"""./data/0-koda-gtfs-rt/tu/{params['date']}.7z"""
    if os.path.exists(file_path_vp) and os.path.exists(file_path_tu):
        continue;

    # Trip Updates
    received = False
    while received == False:
        response_tu = requests.get('https://api.koda.trafiklab.se/KoDa/api/v2/gtfs-rt/sl/TripUpdates', 
                        params=params, headers=headers)
        if response_tu.status_code != 202:
            print(response_tu.json().get("message"))
            received = True
            time.sleep(30)

    if str(response_tu.status_code) == "200":
        with open(file_path_tu, 'wb') as file:
            file.write(response_tu.content)

    # Vehicle Positions
    received = False
    while received == False:
        response_vp = requests.get('https://api.koda.trafiklab.se/KoDa/api/v2/gtfs-rt/sl/VehiclePositions', 
                                params=params, headers=headers)
        if response_tu.status_code != 202:
            print(response_vp.json().get("message"))
            received = True
            time.sleep(30)

    if str(response_vp.status_code) == "200":
        with open(file_path_vp, 'wb') as file:
            file.write(response_vp.content)

In [ ]:
# Downloading static daily historical data.
headers = {'accept': 'application/octet-stream'}

# Loop over all days from 1 to 26 (last one is not included)
for date in tqdm(dates):

    # Create parameter dictionary including key for this iteration
    params = {
        'date': date,
        'key': API_KEY,
    }

    # Construct filename and output path.
    # If file is already downloaded, we skip this iteration.
    file_path = f"""./data/0-koda-gtfs-static/{params['date']}.zip"""
    if os.path.exists(file_path):
        continue;

    # Fetch actual response
    response = requests.get('https://api.koda.trafiklab.se/KoDa/api/v2/gtfs-static/sl', 
                            params=params, headers=headers)

    # If status code is 200 (HTTP OK), save file. 
    if str(response.status_code) == "200":
        with open(file_path, 'wb') as file:
            file.write(response.content)